##### Google Colab Notebook
# DistilBERT
## Inputs: Review

In [1]:
import torch
torch.cuda.is_available()

True

In [2]:
from google.colab import drive

# This will prompt you to click on a link and get an authentication code
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
from tqdm.auto import tqdm
import numpy as np

from sklearn.model_selection import train_test_split
from transformers import TFDistilBertForSequenceClassification, DistilBertTokenizer
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.metrics import SparseCategoricalAccuracy
from tensorflow.keras.callbacks import EarlyStopping

In [4]:
# Pandas Progress bar
tqdm.pandas()

In [5]:
dataset_path = '/content/drive/My Drive/Books_rating.csv'
df = pd.read_csv(dataset_path)

In [6]:
df[['helpful_0', 'helpful_1']] = df['review/helpfulness'].str.split('/', expand=True)

# Konvertieren der 'helpfulness'-Spalte in numerische Werte (falls notwendig)
#df['review/helpfulness'] = pd.to_numeric(df['review/helpfulness'], errors='coerce')

df.head()

,Id,Title,Price,User_id,profileName,review/helpfulness,review/score,review/time,review/summary,review/text,helpful_0,helpful_1
0,1882931173,Its Only Art If Its Well Hung!,NaN,AVCGYZL8FQQTD,"Jim of Oz ""jim-of-oz""",7/7,4.0,940636800,Nice collection of Julie Strain images,This is only for Julie Strain fans. It's a col...,7,7
1,0826414346,Dr. Seuss: American Icon,NaN,A30TK6U7DNS82R,Kevin Killian,10/10,5.0,1095724800,Really Enjoyed It,I don't care much for Dr. Seuss but after read...,10,10
2,0826414346,Dr. Seuss: American Icon,NaN,A3UH4UZ4RSVO82,John Granger,10/11,5.0,1078790400,Essential for every personal and Public Library,"If people become the books they read and if ""t...",10,11
3,0826414346,Dr. Seuss: American Icon,NaN,A2MVUWT453QH61,"Roy E. Perry ""amateur philosopher""",7/7,4.0,1090713600,Phlip Nel gives silly Seuss a serious treatment,"Theodore Seuss Geisel (1904-1991), aka &quot;D...",7,7
4,0826414346,Dr. Seuss: American Icon,NaN,A22X4XUPKF66MR,"D. H. Richards ""ninthwavestore""",3/3,4.0,1107993600,Good academic overview,Philip Nel - Dr. Seuss: American IconThis is b...,3,3


In [7]:
# Daten vor dem Plotten sortieren/konvertieren
df['helpful_0'] = df['helpful_0'].progress_apply(pd.to_numeric)
df['helpful_1'] = df['helpful_1'].progress_apply(pd.to_numeric)

  0%|          | 0/3000000 [00:00<?, ?it/s]

  0%|          | 0/3000000 [00:00<?, ?it/s]

In [8]:
df['helpful_0'] = df['helpful_0'].astype(int)
df['helpful_1'] = df['helpful_1'].astype(int)

In [9]:
num_rows = len(df[df['helpful_1'] >= 20])
print("Number of rows with helpful_1 > 3: ", num_rows)

Number of rows with helpful_1 > 3:  239553


In [10]:
# Remove reviews with not enough helpful votes
df = df[df['helpful_1'] >= 20]

In [11]:
df['helpful_ratio'] = df['helpful_0'] / df['helpful_1']
df['ratio_percent'] = (df['helpful_ratio'] * 100).astype(int)

<ipython-input-11-34f0e5e5852d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['helpful_ratio'] = df['helpful_0'] / df['helpful_1']
<ipython-input-11-34f0e5e5852d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['ratio_percent'] = (df['helpful_ratio'] * 100).astype(int)


In [12]:
# Drop unused columns
df.drop(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/time', 'review/summary'], axis='columns', inplace=True)

<ipython-input-12-4c1a89ba6f12>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(['Id', 'Title', 'Price', 'User_id', 'profileName', 'review/time', 'review/summary'], axis='columns', inplace=True)


In [13]:
df = df.rename(columns={"review/text":"Review","review/score": "Rating"})

df['review_len'] = [len(text.split()) for text in df.Review]

In [14]:
# Cut reviews with less than 300 words
df = df[df['review_len'] <= 300]

In [15]:
def convert_label(df) :
    if df['ratio_percent'] <= 70 :
        rate = 0 # for Negative
    else :
        rate = 1 # for Positive

    return rate

In [24]:
df['IsHelpful'] = df.apply(convert_label, axis = 1)
df.head()

,review/helpfulness,Rating,Review,helpful_0,helpful_1,helpful_ratio,ratio_percent,review_len,IsHelpful
91,36/37,5.0,Ruth Allman has written an excellent book abou...,36,37,0.972973,97,153,1
92,29/30,5.0,"I have been using this book since 1988, the ei...",29,30,0.966667,96,63,1
93,25/28,5.0,"My poor dogeared, stained copy of this book ca...",25,28,0.892857,89,206,1
127,3/20,1.0,This book in my opinion is biased and takes an...,3,20,0.150000,15,92,0
139,20/20,5.0,If you're already a fan of the Eyewitness Trav...,20,20,1.000000,100,234,1


### Data Prepossessing :ghost:

ChatGPT hat bei Preprocessing nichts anderes außer Tokenizing erwähnt, mal schauen wie das läuft

In [17]:
# Tokenize the reviews using DistilBERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
X = tokenizer(df['Review'].tolist(), padding=True, truncation=True, max_length=512, return_tensors='tf')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [18]:
# Convert labels to numpy array
y = df['IsHelpful'].values

# Convert input_ids to numpy array
X_ids = np.array(X['input_ids'].numpy())

In [19]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_ids, y, test_size=0.2, random_state=42)

In [20]:
# Build the DistilBERT model
model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

In [21]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=5e-5),
              loss=SparseCategoricalCrossentropy(from_logits=True),
              metrics=[SparseCategoricalAccuracy('accuracy')])

In [22]:
# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=3,
                    batch_size=8,
                    callbacks=[EarlyStopping(patience=2)])

Epoch 1/3
18111/18111 [==============================] - 3130s 172ms/step - loss: 0.3989 - accuracy: 0.8231 - val_loss: 0.3656 - val_accuracy: 0.8441
Epoch 2/3
18111/18111 [==============================] - 3101s 171ms/step - loss: 0.3084 - accuracy: 0.8703 - val_loss: 0.3435 - val_accuracy: 0.8533
Epoch 3/3
18111/18111 [==============================] - 3103s 171ms/step - loss: 0.2444 - accuracy: 0.9016 - val_loss: 0.4069 - val_accuracy: 0.8575
